In [1]:
import pandas as pd

In [2]:
df_pooled = pd.read_parquet("../data/processed/default_labels/window=12m/pooled.parquet")
df_oos = pd.read_parquet("../data/processed/default_labels/window=12m/oos.parquet")

In [6]:
print(f"Shape of the data :")
print(f"Shape of train+validation :{df_pooled.shape}")
print(f"Shape of OOS :{df_oos.shape}")



print(f"\n Columns :")
print(df_pooled.columns)

Shape of the data :
Shape of train+validation :(27315191, 37)
Shape of OOS :(2512504, 37)

 Columns :
Index(['credit_score', 'first_payment_date', 'first_time_homebuyer_flag',
       'maturity_date', 'msa_md', 'mi_percent', 'number_of_units',
       'occupancy_status', 'original_cltv', 'original_dti', 'original_upb',
       'original_ltv', 'original_interest_rate', 'channel', 'ppm_flag',
       'amortization_type', 'property_state', 'property_type', 'postal_code',
       'loan_sequence_number', 'loan_purpose', 'original_loan_term',
       'number_of_borrowers', 'seller_name', 'servicer_name',
       'super_conforming_flag', 'pre_relief_refi_loan_seq_number',
       'special_eligibility_program', 'relief_refinance_indicator',
       'property_valuation_method', 'interest_only_indicator',
       'mi_cancellation_indicator', 'default_12m', '__file_quarter', 'vintage',
       'window', 'quarter'],
      dtype='object')


In [9]:
import numpy as np
import pandas as pd
from IPython.display import display

# =========================
# Config
# =========================
TARGET = "default_12m"
TIME_COLS = ["quarter", "__file_quarter", "vintage", "window"]
DATE_COLS = ["first_payment_date", "maturity_date"]  # si ces colonnes sont des strings

# =========================
# Helpers
# =========================
def _safe_minmax(series: pd.Series):
    """
    Renvoie (min, max) même si la Series est Categorical non ordonnée.
    On travaille sur une version string et on trie lexicographiquement.
    """
    s = series
    if pd.api.types.is_categorical_dtype(s):
        s = s.astype(str).replace("nan", np.nan)
    s = s.dropna()
    if len(s) == 0:
        return (np.nan, np.nan)
    arr = np.sort(s.astype(str).values)
    return (arr[0], arr[-1])


def portfolio_summary(df: pd.DataFrame, name: str) -> pd.DataFrame:
    out = {"dataset": name, "N": len(df)}

    if TARGET in df.columns:
        s = df[TARGET]
        out["N_defaults"] = int(s.sum(skipna=True))
        out["default_rate"] = float(s.mean(skipna=True))
        out["target_missing_rate"] = float(s.isna().mean())

    for c in TIME_COLS:
        if c in df.columns:
            mn, mx = _safe_minmax(df[c])
            out[f"{c}_min"] = mn
            out[f"{c}_max"] = mx

    return pd.DataFrame([out])


def infer_kind(s: pd.Series) -> str:
    if pd.api.types.is_bool_dtype(s):
        return "bool"
    if pd.api.types.is_numeric_dtype(s):
        return "numeric"
    if pd.api.types.is_datetime64_any_dtype(s):
        return "datetime"
    return "categorical"


def describe_variables(
    df: pd.DataFrame,
    name: str,
    sample_n: int = 1_000_000,
    random_state: int = 0,
    top_k: int = 5,
    q=(0.05, 0.50, 0.95),
) -> pd.DataFrame:
    """
    Retourne un DataFrame "long" : 1 ligne par variable.
    - numeric: mean/std/min/p05/p50/p95/max (sur un échantillon)
    - categorical/bool: top-k modalités + parts (sur un échantillon)
    - time cols: min/max (safe pour categoricals)
    - date cols (si strings): parse en datetime et donne min/p50/max
    """
    # Échantillon pour stats lourdes (quantiles / value_counts)
    df_s = df.sample(n=min(sample_n, len(df)), random_state=random_state) if len(df) > sample_n else df

    rows = []
    for col in df.columns:
        s = df[col]
        kind = infer_kind(s)
        missing = float(s.isna().mean())
        n_unique = int(s.nunique(dropna=True))

        base = {
            "dataset": name,
            "variable": col,
            "type": kind,
            "missing_rate": missing,
            "n_unique": n_unique,
        }

        # Colonnes "temps" : min/max robustes (même si Categorical non ordonné)
        if col in TIME_COLS:
            mn, mx = _safe_minmax(s)
            base["type"] = "time_index"
            base["min"] = mn
            base["max"] = mx
            rows.append(base)
            continue

        # Dates (si stockées en str/object)
        if kind == "categorical" and col in DATE_COLS:
            parsed = pd.to_datetime(df_s[col], errors="coerce")
            base["type"] = "datetime"
            base["min"] = parsed.min()
            base["p50"] = parsed.quantile(0.50)
            base["max"] = parsed.max()
            # missing rate "vrai" après parsing (utile si strings foireuses)
            base["missing_rate"] = float(pd.to_datetime(df[col], errors="coerce").isna().mean())
            rows.append(base)
            continue

        # Numériques
        if kind == "numeric":
            xs = df_s[col].dropna()
            if len(xs) == 0:
                rows.append(base)
                continue
            base.update({
                "mean": float(xs.mean()),
                "std": float(xs.std(ddof=0)),
                "min": float(xs.min()),
                "p05": float(xs.quantile(q[0])),
                "p50": float(xs.quantile(q[1])),
                "p95": float(xs.quantile(q[2])),
                "max": float(xs.max()),
            })
            rows.append(base)
            continue

        # Catégorielles / bool
        xs = df_s[col].dropna()
        if len(xs) == 0:
            rows.append(base)
            continue

        vc = xs.value_counts(dropna=True).head(top_k)
        base["top1"] = vc.index[0]
        base["top1_share"] = float(vc.iloc[0] / len(xs))

        for i in range(top_k):
            if i < len(vc):
                base[f"top{i+1}"] = vc.index[i]
                base[f"top{i+1}_share"] = float(vc.iloc[i] / len(xs))

        rows.append(base)

    desc = pd.DataFrame(rows)

    # Ordre de colonnes lisible
    preferred = [
        "dataset", "variable", "type", "missing_rate", "n_unique",
        "mean", "std", "min", "p05", "p50", "p95", "max",
        "top1", "top1_share", "top2", "top2_share", "top3", "top3_share",
        "top4", "top4_share", "top5", "top5_share",
    ]
    cols = [c for c in preferred if c in desc.columns] + [c for c in desc.columns if c not in preferred]
    desc = desc[cols].sort_values(["type", "missing_rate", "n_unique"], ascending=[True, False, False])

    return desc


# =========================
# Usage (df_pooled, df_oos doivent exister)
# =========================
port = pd.concat(
    [
        portfolio_summary(df_pooled, "train+val"),
        portfolio_summary(df_oos, "oos"),
    ],
    ignore_index=True
)
display(port)

desc_pooled = describe_variables(df_pooled, "train+val", sample_n=1_000_000, random_state=0, top_k=5)
desc_oos = describe_variables(df_oos, "oos", sample_n=1_000_000, random_state=0, top_k=5)

display(desc_pooled)
display(desc_oos)

# (option) exports CSV
# desc_pooled.to_csv("descriptives_train_val.csv", index=False)
# desc_oos.to_csv("descriptives_oos.csv", index=False)


/var/folders/yk/g149s5cs3m1f2t6smf32gc1m0000gn/T/ipykernel_5385/3428177318.py:21: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(s):
/var/folders/yk/g149s5cs3m1f2t6smf32gc1m0000gn/T/ipykernel_5385/3428177318.py:21: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(s):
/var/folders/yk/g149s5cs3m1f2t6smf32gc1m0000gn/T/ipykernel_5385/3428177318.py:21: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(s):
/var/folders/yk/g149s5cs3m1f2t6smf32gc1m0000gn/T/ipykernel_5385/3428177318.py:21: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use is

,dataset,N,N_defaults,default_rate,target_missing_rate,quarter_min,quarter_max,__file_quarter_min,__file_quarter_max,vintage_min,vintage_max,window_min,window_max
0,train+val,27315191,164440,0.006020,0.0,2008Q1,2021Q4,2008Q1,2021Q4,2008Q1,2021Q4,12m,12m
1,oos,2512504,18541,0.007379,0.0,2022Q1,2023Q4,2022Q1,2023Q4,2022Q1,2023Q4,12m,12m


/var/folders/yk/g149s5cs3m1f2t6smf32gc1m0000gn/T/ipykernel_5385/3428177318.py:102: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df_s[col], errors="coerce")
/var/folders/yk/g149s5cs3m1f2t6smf32gc1m0000gn/T/ipykernel_5385/3428177318.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  base["missing_rate"] = float(pd.to_datetime(df[col], errors="coerce").isna().mean())
/var/folders/yk/g149s5cs3m1f2t6smf32gc1m0000gn/T/ipykernel_5385/3428177318.py:21: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(s):
/var/folders/yk/g149s5cs3m1f2t6smf32gc1m0000gn/T/ipyk

,dataset,variable,type,missing_rate,n_unique,mean,std,min,p05,p50,...,top1,top1_share,top2,top2_share,top3,top3_share,top4,top4_share,top5,top5_share
25,train+val,super_conforming_flag,categorical,9.668151e-01,1,NaN,NaN,NaN,NaN,NaN,...,Y,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,train+val,pre_relief_refi_loan_seq_number,categorical,8.950950e-01,2864814,NaN,NaN,NaN,NaN,NaN,...,F09Q10282011,0.000019,F02Q30553035,0.000019,F07Q30269519,0.000010,F05Q10177878,0.000010,F02Q41148654,0.000010
28,train+val,relief_refinance_indicator,categorical,8.950950e-01,1,NaN,NaN,NaN,NaN,NaN,...,Y,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,train+val,servicer_name,categorical,7.321933e-08,66,NaN,NaN,NaN,NaN,NaN,...,Other servicers,0.274813,"WELLS FARGO BANK, N.A.",0.153399,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",0.066377,U.S. BANK N.A.,0.062538,"BANK OF AMERICA, N.A.",0.037235
19,train+val,loan_sequence_number,categorical,0.000000e+00,27315191,NaN,NaN,NaN,NaN,NaN,...,F21Q30006744,0.000001,F09Q10595336,0.000001,F20Q10069752,0.000001,F08Q20025525,0.000001,F19Q20368471,0.000001
18,train+val,postal_code,categorical,0.000000e+00,896,NaN,NaN,NaN,NaN,NaN,...,94500,0.012132,75000,0.009807,30000,0.009399,84000,0.008957,85200,0.008803
23,train+val,seller_name,categorical,0.000000e+00,78,NaN,NaN,NaN,NaN,NaN,...,Other sellers,0.310029,"WELLS FARGO BANK, N.A.",0.153202,U.S. BANK N.A.,0.056291,"JPMORGAN CHASE BANK, N.A.",0.039728,"BANK OF AMERICA, N.A.",0.037252
16,train+val,property_state,categorical,0.000000e+00,54,NaN,NaN,NaN,NaN,NaN,...,CA,0.137265,TX,0.061766,FL,0.055137,IL,0.049638,OH,0.036125
13,train+val,channel,categorical,0.000000e+00,5,NaN,NaN,NaN,NaN,NaN,...,R,0.596589,C,0.281980,B,0.110340,T,0.011089,9,0.000002
17,train+val,property_type,categorical,0.000000e+00,5,NaN,NaN,NaN,NaN,NaN,...,SF,0.676219,PU,0.245294,CO,0.072982,MH,0.003726,CP,0.001779


,dataset,variable,type,missing_rate,n_unique,mean,std,min,p05,p50,...,top1,top1_share,top2,top2_share,top3,top3_share,top4,top4_share,top5,top5_share
26,oos,pre_relief_refi_loan_seq_number,categorical,1.000000,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,oos,relief_refinance_indicator,categorical,1.000000,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,oos,super_conforming_flag,categorical,0.987080,1,NaN,NaN,NaN,NaN,NaN,...,Y,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,oos,loan_sequence_number,categorical,0.000000,2512504,NaN,NaN,NaN,NaN,NaN,...,F22Q30263415,0.000001,F22Q30187545,0.000001,F23Q30164541,0.000001,F23Q20130598,0.000001,F23Q40050601,0.000001
18,oos,postal_code,categorical,0.000000,891,NaN,NaN,NaN,NaN,NaN,...,75000,0.011852,30000,0.010623,85300,0.007885,78600,0.007874,60600,0.007498
16,oos,property_state,categorical,0.000000,54,NaN,NaN,NaN,NaN,NaN,...,TX,0.096074,FL,0.088224,CA,0.079243,IL,0.041559,OH,0.040729
23,oos,seller_name,categorical,0.000000,31,NaN,NaN,NaN,NaN,NaN,...,Other sellers,0.319798,"UNITED WHOLESALE MORTGAGE, LLC",0.104802,"ROCKET MORTGAGE, LLC",0.074962,"AMERIHOME MORTGAGE COMPANY, LLC",0.055701,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",0.048536
24,oos,servicer_name,categorical,0.000000,30,NaN,NaN,NaN,NaN,NaN,...,Other servicers,0.235340,"LAKEVIEW LOAN SERVICING, LLC",0.088275,NATIONSTAR MORTGAGE LLC DBA MR. COOPER,0.081901,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",0.064197,"ROCKET MORTGAGE, LLC",0.061238
17,oos,property_type,categorical,0.000000,5,NaN,NaN,NaN,NaN,NaN,...,SF,0.605279,PU,0.293092,CO,0.088089,MH,0.011115,CP,0.002425
13,oos,channel,categorical,0.000000,4,NaN,NaN,NaN,NaN,NaN,...,R,0.517758,C,0.328277,B,0.153965,NaN,NaN,NaN,NaN


In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display

# =========================
# Choix du dataset & variables
# =========================
df = df_pooled  # ou df_oos
TARGET = "default_12m"

NUM_VARS = [
    "credit_score",
    "mi_percent",
    "number_of_units",
    "original_cltv",
    "original_dti",
    "original_upb",
    "original_ltv",
    "original_interest_rate",
    "original_loan_term",
    "number_of_borrowers",
]

BIN_VARS = [
    "first_time_homebuyer_flag",
    "ppm_flag",
    "super_conforming_flag",
    "relief_refinance_indicator",
    "interest_only_indicator",
    "mi_cancellation_indicator",
    TARGET,
]

LABELS = {
    "credit_score": "Credit score",
    "mi_percent": "MI percent",
    "number_of_units": "Number of units",
    "original_cltv": "Original CLTV",
    "original_dti": "Original DTI",
    "original_upb": "Original UPB",
    "original_ltv": "Original LTV",
    "original_interest_rate": "Original interest rate",
    "original_loan_term": "Original loan term",
    "number_of_borrowers": "Number of borrowers",
    "first_time_homebuyer_flag": "First-time homebuyer (0/1)",
    "ppm_flag": "PPM flag (0/1)",
    "super_conforming_flag": "Super conforming (0/1)",
    "relief_refinance_indicator": "Relief refinance (0/1)",
    "interest_only_indicator": "Interest-only (0/1)",
    "mi_cancellation_indicator": "MI cancellation (0/1)",
    TARGET: "Default (0/1)",
}

NUM_VARS = [c for c in NUM_VARS if c in df.columns]
BIN_VARS = [c for c in BIN_VARS if c in df.columns]
VARS = NUM_VARS + [c for c in BIN_VARS if c not in NUM_VARS]


# =========================
# Helpers: binaire robuste
# =========================
_TRUE = {"1", "y", "yes", "t", "true"}
_FALSE = {"0", "n", "no", "f", "false"}

def to_binary_series(s: pd.Series) -> pd.Series | None:
    """
    Essaie de convertir une série en {0,1} si elle est binaire.
    Retourne une Series float (0/1) ou None si pas binaire.
    """
    x = s.dropna()

    if len(x) == 0:
        return pd.Series([], dtype=float)

    # bool natif
    if pd.api.types.is_bool_dtype(x):
        return x.astype(float)

    # numérique
    if pd.api.types.is_numeric_dtype(x):
        ux = pd.unique(x)
        ux = ux[~pd.isna(ux)]
        # on tolère {0,1} uniquement
        try:
            vals = set(np.unique(pd.to_numeric(pd.Series(ux), errors="coerce").dropna().astype(float)))
        except Exception:
            vals = set()
        if vals <= {0.0, 1.0} and x.nunique() <= 2:
            return pd.to_numeric(s, errors="coerce").astype(float)

    # strings / categoricals
    xs = x.astype(str).str.strip().str.lower()
    u = set(pd.unique(xs))
    if u <= (_TRUE | _FALSE) and len(u) <= 2:
        mapped = xs.map(lambda v: 1.0 if v in _TRUE else 0.0)
        # remettre les NaN d'origine
        out = pd.Series(np.nan, index=s.index, dtype=float)
        out.loc[x.index] = mapped.values
        return out

    return None


def make_paper_descriptives(
    df: pd.DataFrame,
    variables: list[str],
    labels: dict[str, str] | None = None,
    sample_n: int = 1_000_000,
    random_state: int = 0,
) -> pd.DataFrame:
    """
    Table style papier:
    Variable | Count | Mean | Std. Dev. | Min | 25% | Median | 75% | Max

    - Numériques: stats complètes (quantiles sur échantillon)
    - Binaires (0/1, bool, Y/N, etc.): Mean + Min/Max, et '–' pour Std/quantiles
    """
    labels = labels or {}

    # échantillon pour quantiles (rapide)
    df_s = df.sample(n=min(sample_n, len(df)), random_state=random_state) if len(df) > sample_n else df

    rows = []
    for col in variables:
        s_full = df[col]
        s_samp = df_s[col]

        count = int(s_full.notna().sum())

        # tentative binaire
        b_full = to_binary_series(s_full)
        b_samp = to_binary_series(s_samp)

        is_binary = b_samp is not None and (b_samp.dropna().nunique() <= 2)

        if is_binary:
            mean = float(b_full.mean(skipna=True)) if b_full is not None else float(b_samp.mean(skipna=True))
            row = {
                "Variable": labels.get(col, col),
                "Count": count,
                "Mean": mean,
                "Std. Dev.": "–",
                "Min": 0.0,
                "25%": "–",
                "Median": "–",
                "75%": "–",
                "Max": 1.0,
            }
            rows.append(row)
            continue

        # sinon: numérique
        xs_num = pd.to_numeric(s_samp, errors="coerce").dropna()
        mean_full = float(pd.to_numeric(s_full, errors="coerce").mean(skipna=True))

        if len(xs_num) == 0:
            row = {
                "Variable": labels.get(col, col),
                "Count": count,
                "Mean": mean_full,
                "Std. Dev.": "–",
                "Min": "–",
                "25%": "–",
                "Median": "–",
                "75%": "–",
                "Max": "–",
            }
        else:
            q25, q50, q75 = xs_num.quantile([0.25, 0.50, 0.75]).tolist()
            row = {
                "Variable": labels.get(col, col),
                "Count": count,
                "Mean": mean_full,
                "Std. Dev.": float(xs_num.std(ddof=0)),
                "Min": float(xs_num.min()),
                "25%": float(q25),
                "Median": float(q50),
                "75%": float(q75),
                "Max": float(xs_num.max()),
            }

        rows.append(row)

    out = pd.DataFrame(rows)

    # formatting "paper"
    def _fmt(x):
        if isinstance(x, str):
            return x
        if pd.isna(x):
            return "–"
        return f"{x:.3f}"

    out["Count"] = out["Count"].astype(int)
    for c in ["Mean", "Std. Dev.", "Min", "25%", "Median", "75%", "Max"]:
        out[c] = out[c].map(_fmt)

    return out


# =========================
# Run
# =========================
table_pooled = make_paper_descriptives(df, VARS, labels=LABELS, sample_n=1_000_000, random_state=0)
display(table_pooled)

table_oos = make_paper_descriptives(df_oos, VARS, labels=LABELS, sample_n=1_000_000, random_state=0)
display(table_oos)


# Option: export
# table.to_csv("descriptive_stats_table.csv", index=False)
# print(table.to_latex(index=False, escape=True))

,Variable,Count,Mean,Std. Dev.,Min,25%,Median,75%,Max
0,Credit score,27315191,754.676,121.519,333.000,724.000,764.000,790.000,9999.000
1,MI percent,27315191,4.777,10.267,0.000,0.000,0.000,0.000,999.000
2,Number of units,27315191,1.034,0.907,1.000,1.000,1.000,1.000,99.000
3,Original CLTV,27315191,72.887,20.540,1.000,61.000,75.000,83.000,999.000
4,Original DTI,27315191,135.969,296.710,1.000,27.000,36.000,44.000,999.000
5,Original UPB,27315191,239480.653,130918.098,10000.000,140000.000,214000.000,317000.000,1500000.000
6,Original LTV,27315191,71.808,19.931,1.000,60.000,75.000,80.000,999.000
7,Original interest rate,27315191,3.965,0.923,1.500,3.250,3.875,4.625,8.750
8,Original loan term,27315191,314.691,76.278,60.000,240.000,360.000,360.000,559.000
9,Number of borrowers,27315191,1.531,0.762,1.000,1.000,2.000,2.000,99.000


,Variable,Count,Mean,Std. Dev.,Min,25%,Median,75%,Max
0,Credit score,2512504,748.340,156.336,513.000,714.000,754.000,784.000,9999.000
1,MI percent,2512504,8.701,12.787,0.000,0.000,0.000,25.000,35.000
2,Number of units,2512504,1.032,0.230,1.000,1.000,1.000,1.000,4.000
3,Original CLTV,2512504,74.263,18.872,3.000,64.000,80.000,90.000,184.000
4,Original DTI,2512504,37.322,13.904,1.000,31.000,39.000,45.000,999.000
5,Original UPB,2512504,303816.660,165578.431,10000.000,179000.000,271000.000,399000.000,1988000.000
6,Original LTV,2512504,74.103,18.830,3.000,64.000,80.000,90.000,184.000
7,Original interest rate,2512504,5.476,1.419,1.750,4.250,5.625,6.625,9.750
8,Original loan term,2512504,341.656,53.216,85.000,360.000,360.000,360.000,366.000
9,Number of borrowers,2512504,1.456,0.533,1.000,1.000,1.000,2.000,5.000
